#LinUCB contextual bandit algorithm
**is used here to create an adaptive learning environment
as learners improve**

In [ ]:
import numpy as np
import logging
import pickle

# Configure logging
logging.basicConfig(filename='linucb.log', level=logging.INFO, format='%(asctime)s - %(message)s')


def save_context(context, filename="context.pkl"):
    with open(filename, "wb") as f:
        pickle.dump(context, f)

def load_context(filename="context.pkl"):
    try:
        with open(filename, "rb") as f:
            return pickle.load(f)
    except FileNotFoundError:
        return None


# List of subjects
subjects = [
    'contest 1', 'contest 2', 'contest 3', 'contest 4', 'contest 5', 'contest 6', 'contest 7', 'contest 8', 'contest 9', 'contest 10',
    'contest 11', 'contest 12', 'contest 13', 'contest 14', 'contest 15', 'contest 16', 'contest 17', 'contest 18', 'contest 19', 'contest 20',
    'contest 21', 'contest 22', 'contest 23', 'contest 24', 'contest 25', 'contest 26', 'contest 27', 'contest 28', 'contest 29', 'contest 30',
    'contest 31', 'contest 32', 'contest 33', 'contest 34', 'contest 35', 'contest 36', 'contest 37', 'contest 38', 'contest 39', 'contest 40'
]


class LinUCB:
    def __init__(self, n_actions, n_features, alpha=1.0):
        self.n_actions = n_actions
        self.n_features = n_features
        self.alpha = alpha

        # Initialize parameters
        self.A = np.array([np.identity(n_features) for _ in range(n_actions)])  # action covariance matrix
        self.b = np.array([np.zeros(n_features) for _ in range(n_actions)])  # action reward vector
        self.theta = np.array([np.zeros(n_features) for _ in range(n_actions)])  # action parameter vector

        # Initialize interaction counts
        self.interaction_counts = np.zeros(n_actions)

    def predict(self, context):
        context = np.array(context)  # Convert list to ndarray
        p = np.zeros(self.n_actions)
        for a in range(self.n_actions):
            self.theta[a] = np.dot(np.linalg.inv(self.A[a]), self.b[a])  # theta_a = A_a^-1 * b_a
            p[a] = np.dot(self.theta[a], context) + self.alpha * np.sqrt(np.dot(context, np.dot(np.linalg.inv(self.A[a]), context)))
        return p

    def update(self, action, context, reward):
        context = np.array(context)  # Convert list to ndarray if necessary
        context = context.reshape(-1)  # Ensure context is a flat array
        self.A[action] += np.outer(context, context)  # A_a = A_a + x_t * x_t^T
        self.b[action] += reward * context  # b_a = b_a + r_t * x_t
        self.interaction_counts[action] += 1  # Increment interaction count for the chosen action

        # Log the update
        logging.info(f"Action: {action}, Context: {context.tolist()}, Reward: {reward}")
        self.save_state()

    def save_state(self, filename="model_state.pkl"):
        with open(filename, "wb") as f:
            pickle.dump(self, f)

    @staticmethod
    def load_state(filename="model_state.pkl"):
        try:
            with open(filename, "rb") as f:
                return pickle.load(f)
        except FileNotFoundError:
            return None


# Example usage

# Suppose we have 40 subjects (actions) and each context is a 4-dimensional feature vector
n_actions = len(subjects)
n_features = 1
alpha = 1.0

# Try to load an existing model, otherwise initialize a new one
model = LinUCB.load_state() or LinUCB(n_actions, n_features, alpha)

# Example context vector for a user (e.g., user's preferences or history in some feature space)
context = [1]


# Predict the preference scores for each subject
preference_scores = model.predict(context)
print("Preference scores:", preference_scores)

# Select the action (subject) with the highest score
chosen_action = np.argmax(preference_scores)
print("Chosen subject based on preference:", subjects[chosen_action])

# print(type(subjects.index(subjects[chosen_action])))
# Update the model with the chosen action, context, and reward (e.g., user clicked on the subject)
answer = "correct"
if answer == "incorrect":
    reward = 1
if answer == "correct":
    reward = 0
model.update(chosen_action, context, reward)

Preference scores: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Chosen subject based on preference: contest 1


This section utilizes Faiss for
#Semantic search,
leveraging the "all-mpnet-base-v2" model as the sentence transformer.

In [ ]:
!pip install -q datasets sentence-transformers faiss-cpu accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 74.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-c

In [ ]:
from datasets import load_dataset, DatasetDict, Dataset
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import os


In [ ]:
dataset = load_dataset('csv', data_files='/content/combined_sheets.csv')

ST = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
sheet_name_mapping = {name: idx for idx, name in enumerate(set(dataset['train']['Subject']).union(set(dataset['train']['Quiz Round']).union(set(dataset['train']['Year_Round']))))}

In [ ]:
# Function to embed the data and include criterion information
def embed_with_criterion(batch):

    information = [q if not p else p + " " + q for p, q in zip(batch["Preamble Text"], batch["Question"])]   # Adjust the column names if necessary
    embeddings = ST.encode(information)

    # Add criterion information to embeddings using the mapping
    criterion_Round = np.array([sheet_name_mapping[name] for name in batch["Subject"]], dtype=np.float32).reshape(-1, 1)
    criterion_Subject = np.array([sheet_name_mapping[name] for name in batch["Quiz Round"]], dtype=np.float32).reshape(-1, 1)
    criterion_year = np.array([sheet_name_mapping[name] for name in batch["Year_Round"]], dtype=np.float32).reshape(-1, 1)
    modified_embeddings = np.hstack((embeddings, criterion_Round,  criterion_Subject, criterion_year))

    return {"embeddings": modified_embeddings}

# Apply the embedding function to the dataset
dataset = dataset.map(embed_with_criterion, batched=True, batch_size=16)

Map:   0%|          | 0/15850 [00:00<?, ? examples/s]

In [ ]:
# Save the dataset and FAISS index locally
save_path = '/content/n_embedded_dataset'
os.makedirs(save_path, exist_ok=True)
dataset.save_to_disk(save_path)
dataset["train"].add_faiss_index(column="embeddings")
dataset["train"].save_faiss_index("embeddings", save_path + '/faiss_index')

dataset = DatasetDict.load_from_disk(save_path)
dataset["train"].load_faiss_index("embeddings", save_path + '/faiss_index')

Saving the dataset (0/1 shards):   0%|          | 0/15850 [00:00<?, ? examples/s]

  0%|          | 0/16 [00:00<?, ?it/s]

In [ ]:
# Function to search for the most similar entries considering the criterion
def search_with_criterion(query: str, k: int = 4, round_value="Round 1", rank_year_value=2021, subject_value="Chemistry"):
    """A function that embeds a new query and returns the most probable results considering the criterion"""
    embedded_query = ST.encode(query)  # Embed new query

    criterion_Round = np.array([sheet_name_mapping[round_value]], dtype=np.float32).reshape(1, 1)
    criterion_Subject = np.array([sheet_name_mapping[subject_value]], dtype=np.float32).reshape(1, 1)
    criterion_year = np.array([sheet_name_mapping[rank_year_value]], dtype=np.float32).reshape(1, 1)
    modified_query_embedding = np.hstack((embedded_query.reshape(1, -1), criterion_Round, criterion_Subject, criterion_year))

    # Retrieve results
    scores, retrieved_examples = dataset["train"].get_nearest_examples(
        "embeddings", modified_query_embedding, k=k
    )

    return scores, retrieved_examples
round_value= "Round 1"
subject_value= "Chemistry"
year_value= "2021"
rank_value= "contest 5"
# Example usage
query = "experiment to determine the acceleration due to gravity"
scores, retrieved_examples = search_with_criterion(query, k=5, round_value= round_value,
                                                   subject_value= subject_value, rank_year_value= f"{year_value} NSMQ {rank_value}")
print(scores)
print(retrieved_examples["Question"])

[10954.853 10954.907 10954.959 10958.925 10958.976]
['A committee of 3 is to be formed from 3 men and 3 women. In how ways can this be done if there are 2 women and 1 man on the committee', "Find the solution set of the equation $|x|=-x$\n(Read as 'absolute value of $x=-x$ ')", 'Describe the set of points $(x, y)$ such that $4<x^{2}+y^{2}<9$', 'Find the equation of the tangent to the curve $y=x^{2}-3 x+2$ at the point on the curve at which $\\mathrm{x}=2$', 'Given that a sequence is defined by $U_{n+2}=U_{n+1}+U_{n}, U_{1}=2, U_{2}=3$, find $U_{3}$.']


# LLM access

In [1]:
import warnings
warnings.filterwarnings('ignore')

import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install -q accelerate bitsandbytes
!pip install -q oauth2client pypdf sentence_transformers
!pip install -q transformers einops accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.1 MB/s eta 0:00:00


In [2]:
!pip install -q huggingface_hub
from huggingface_hub import notebook_login
notebook_login()

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

model_id = "mistralai/Mistral-7B-Instruct-v0.3"

# use quantization to lower GPU usage
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    quantization_config=bnb_config
)


tokenizer_config.json:   0%|          | 0.00/137k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [4]:
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids(["<|eot_id|>", "<|im_end|>"])
]

Round_number= [
    {
        "Round 1": "Fundamentals on Biology, Physics, Chemistry and Maths where each team receives 2/3 sets of questions depending on the stage of the competition you're in.",
        "Round 2": "Speed Race. Quick successive questioning to user and you have to answer a question as quickly and with no delay in providing answers.",
        "Round 3": "Problem of the Day. A question is posed to all three schools and given 3 mins to provide and answer to it.",
        "Round 4": "True/False. Each subject has 2 sets of questions to be answered",
        "Round 5": "Riddles. Each subject has a riddle to answer."
     }
]

In [5]:
def FULL_Quize(query):

    explain_prompt = f"""system

        Your main purpose is to make sure the user understands CONCEPTS

        These are your past conversations

        Your output should always be in the provided JSON fomart
            -fill the most apporpriate field
            -if apporpriate data doesn't exist in the new query put "N/A" at the space

            {{
                    "explain": "put the answer to the users questions here. Break your explainations down to help the user undersatnd well"
                }}
        End immidetly afer this
        <|im_end|>
        Edge cases you must handle:
        - If the user request has completely nothing to do with NSMQ, you will respond politely that you cannot help.<|im_end|>
        """


    ready_prompt = f"""system

    Your have three main purpose:
     1.Find out if the user is ready for the QUESTIONS TO BE ASKED
     2.Explain any thing they don't understand about the quiz


    You will be asking a student certain questions
    In which the student would have different time durations to answer the questions

    "Round 1"-> "questions time frame 30s", "Fundamentals on Biology, Physics", Chemistry and Maths where each team receives 2/3 sets of questions depending on the stage of the competition you're in.",
    "Round 2"-> "questions time frame must be answered as soon as possible", "Speed Race. Quick successive questioning to user and you have to answer a question as quickly and with no delay in providing answers.",
    "Round 3"-> "questions time frame 30s", "Problem of the Day. A question is posed to all three schools and given 3 mins to provide and answer to it.",
    "Round 4"-> "questions time frame 30s", "True/False. Each subject has 2 sets of questions to be answered",
    "Round 5"-> "questions time frame 60s", "Riddles. Each subject has a riddle to answer."

    I have given you all the information to answer user's prompts
    Your goal is to find out if the user is ready to ba quized


    Your output should always be in the provided JSON fomart
        -fill the most apporpriate field
        -if apporpriate data doesn't exist in the new query put "N/A" at the space

           {{
                "isReady": "leave this space as "True" if the user confirms to be ready for the questions"
                "pre-test": "put the answer to the users questions here"
            }}
    End immidetly afer this
    <|im_end|>
    Edge cases you must handle:
    - If the user request has completely nothing to do with NSMQ, you will respond politely that you cannot help.<|im_end|>
    """

   # Call the LLM to get the completion
    return templ_prompt


In [6]:
ASSISTANT = """
I am an assistant for high school students in Ghana.
And my goal is to help them preparing effectively for the National Science and Math Quize(NSMQ).
"""
# If you don't know the answer, just say "I do not know." Don't make up an answer.

def General_prompt(query):

    templ_prompt = f"""system

    From the query provided
    {query}

    extract relevant data from it

    NOTE: NEVER GENERATE ANY SCIENCE QUESTIONS ON YOUR OWN eg "What is matter?" RATHER you can ask questions on the specificity of SCIENCE QUESTIONS eg. What (topic)would you like get questions from?
    Your output should always be in the provided JSON fomart
        -fill the most apporpriate field below
        -if apporpriate data doesn't exist in the new query put "N/A" at the space

           {{
                "quiz": {{
                    "isAsk": "leave this space as "True" ONLY if the user wants to be asked an NSMQ/science question",
                    "query": "put keywords form the query here eg. what is matter? keywords: matter "
                }},
                "year": "Put the YEAR the query here",
                "suject": "Put the SUBJECT in the query here",
                "round": "space specific Round Number:{Round_number} sure to put only the round and its number here eg. Round 1",
                "general_Q": "Put the answer to question the user ASKED here"
            }}
    """
    # Edge cases you must handle:
    # - If the user request has completely nothing to do with NSMQ, you will respond politely that you cannot help.<|im_end|>
   # Call the LLM to get the completion
    return templ_prompt


In [7]:
# Define a prompt
prompt = "what is ur purpose"
# generate(formatted_prompt(prompt))

# def generate(formatted_prompt):
# formatted_prompt = formatted_prompt[:2000] # to avoid GPU OOM
messages = [{"role":"user","content":"hello"},
              {"role":"assistant","content":ASSISTANT},
              {"role":"user","content":General_prompt(prompt)}
            ]
  # tell the model to generate
input_ids = tokenizer.apply_chat_template(
   messages,
   return_tensors="pt"
).to(model.device)
outputs = model.generate(
    input_ids,
    max_new_tokens=1024,
    eos_token_id=tokenizer.eos_token_id,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
# response = outputs[0][input_ids.shape[-1]:]
# tokenizer.decode(response, skip_special_tokens=True)
# print(tokenizer.decode(response, skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


KeyboardInterrupt: 

In [8]:
def general_llm(prompt):
  prompt = prompt
  # Define a prompt
  # prompt = "what is ur purpose"
  # generate(formatted_prompt(prompt))

  messages = [{"role":"user","content":"hello"},
                {"role":"assistant","content":ASSISTANT},
                {"role":"user","content":General_prompt(prompt)}
              ]
    # tell the model to generate
  input_ids = tokenizer.apply_chat_template(
    messages,
    return_tensors="pt"
  ).to(model.device)
  outputs = model.generate(
      input_ids,
      max_new_tokens=1024,
      eos_token_id=tokenizer.eos_token_id,
      pad_token_id=tokenizer.eos_token_id,
      do_sample=True,
      temperature=0.6,
      top_p=0.9,
  )

  response = outputs[0][input_ids.shape[-1]:]
  tokenizer.decode(response, skip_special_tokens=True)

  return(tokenizer.decode(response, skip_special_tokens=True))

In [60]:
print(general_llm("what is ur purpose"))

{
        "quiz": {
            "isAsk": "False",
            "query": "what is ur purpose"
        },
        "year": "N/A",
        "subject": "N/A",
        "round": "N/A",
        "general_Q": "The purpose of this assistant is to help high school students in Ghana preparing effectively for the National Science and Math Quize(NSMQ)"
    }


In [ ]:
import json

In [ ]:
if gen_llm["quiz"]["isAsk"] and (gen_llm["year"] or gen_llm["round"] or gen_llm["subject"]) == True:
  print("yes")


# API access point


In [10]:
# Install the required packages
!pip -q install fastapi uvicorn pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [11]:
!ngrok authtoken 2gtKp9Zgztrv5dtK9SGsMl0cad7_5ZsXKxvA8cP6zvv4WZT94

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [15]:

from fastapi import FastAPI, HTTPException
from pyngrok import ngrok
import uvicorn
from threading import Thread

# Define the FastAPI app
app = FastAPI()

@app.get("/gen_llm/{prompt}")
def read_root(prompt):
    result = general_llm(prompt)
    return result

# Function to run the FastAPI app with Uvicorn
def run_app():
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Start the FastAPI app in a new thread
server_thread = Thread(target=run_app)
server_thread.start()

!killall ngrok
# Expose the FastAPI app with ngrok
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

INFO:     Started server process [254]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('0.0.0.0', 8000): address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


ngrok: no process found
Public URL: NgrokTunnel: "https://d5c7-35-227-177-37.ngrok-free.app" -> "http://localhost:8000"
